# DiffEqBiological Tutorial I: Introduction
### Samuel Isaacson

DiffEqBiological.jl is a domain specific language (DSL) for writing chemical
reaction networks in Julia. The generated chemical reaction network model can
then be translated into a variety of mathematical models which can be solved
using components of the broader
[DifferentialEquations.jl](http://juliadiffeq.org/) ecosystem.

In this tutorial we'll provide an introduction to using DiffEqBiological to
specify chemical reaction networks, and then to solve ODE, jump, tau-leaping and
SDE models generated from them. Let's start by using the DiffEqBiological
`reaction_network` macro to specify a simply chemical reaction network; the
well-known Repressilator. 

We first import the basic packages we'll need, and use Plots.jl for making
figures:

In [ ]:
# If not already installed, first hit "]" within a Julia REPL. Then type:
# add DifferentialEquations DiffEqBiological PyPlot Plots Latexify 

using DifferentialEquations, DiffEqBiological, Plots, Latexify
pyplot(fmt=:svg);

We now construct the reaction network. The basic types of arrows and predefined
rate laws one can use are discussed in detail within the DiffEqBiological
[Chemical Reaction Models
documentation](http://docs.juliadiffeq.org/latest/models/biological.html). Here
we use a mix of first order, zero order and repressive Hill function rate laws.
Note, $\varnothing$ corresponds to the empty state, and is used for zeroth order
production and first order degradation reactions:

In [ ]:
repressilator = @reaction_network begin
    hillr(P₃,α,K,n), ∅ --> m₁
    hillr(P₁,α,K,n), ∅ --> m₂
    hillr(P₂,α,K,n), ∅ --> m₃
    (δ,γ), m₁ ↔ ∅
    (δ,γ), m₂ ↔ ∅
    (δ,γ), m₃ ↔ ∅
    β, m₁ --> m₁ + P₁
    β, m₂ --> m₂ + P₂
    β, m₃ --> m₃ + P₃
    μ, P₁ --> ∅
    μ, P₂ --> ∅
    μ, P₃ --> ∅
end α K n δ γ β μ;

We can use Latexify to look at the corresponding reactions and understand the
generated rate laws for each reaction

In [ ]:
latexify(repressilator; env=:chemical)

We can also use Latexify to look at the corresponding ODE model for the chemical
system

In [ ]:
latexify(repressilator, cdot=false)

To solve the ODEs we need to specify the values of the parameters in the model,
the initial condition, and the time interval to solve the model on. To do this
it helps to know the orderings of the parameters and the species. Parameters are
ordered in the same order they appear after the `end` statement in the
`@reaction_network` macro. Species are ordered in the order they first appear
within the `@reaction_network` macro. We can see these orderings using the
`speciesmap` and `paramsmap` functions:

In [ ]:
speciesmap(repressilator)

In [ ]:
paramsmap(repressilator)

## Solving the ODEs:
Knowing these orderings, we can create parameter and initial condition vectors,
and setup the `ODEProblem` we want to solve:

In [ ]:
# parameters [α,K,n,δ,γ,β,μ]
p = (.5, 40, 2, log(2)/120, 5e-3, 20*log(2)/120, log(2)/60)

# initial condition [m₁,m₂,m₃,P₁,P₂,P₃]
u₀ = [0.,0.,0.,20.,0.,0.]

# time interval to solve on
tspan = (0., 10000.)

# create the ODEProblem we want to solve
oprob = ODEProblem(repressilator, u₀, tspan, p)

At this point we are all set to solve the ODEs. We can now use any ODE solver
from within the DiffEq package. We'll just use the default DifferentialEquations
solver for now, and then plot the solutions:

In [ ]:
sol = solve(oprob, saveat=10.)
plot(sol, fmt=:svg)

We see the well-known oscillatory behavior of the repressilator! For more on
choices of ODE solvers, see the JuliaDiffEq
[documentation](http://docs.juliadiffeq.org/latest/solvers/ode_solve.html).

---

## Stochastic Simulation Algorithms (SSAs) for Stochastic Chemical Kinetics
Let's now look at a stochastic chemical kinetics model of the repressilator,
modeling it with jump processes. Here we will construct a DiffEqJump
`JumpProblem` that uses Gillespie's `Direct` method, and then solve it to
generate one realization of the jump process:

In [ ]:
# first we redefine the initial condition to be integer valued
u₀ = [0,0,0,20,0,0]

# next we create a discrete problem to encode that our species are integer valued:
dprob = DiscreteProblem(repressilator, u₀, tspan, p)

# now we create a JumpProblem, and specify Gillespie's Direct Method as the solver:
jprob = JumpProblem(dprob, Direct(), repressilator, save_positions=(false,false))

# now let's solve and plot the jump process:
sol = solve(jprob, SSAStepper(), saveat=10.)
plot(sol, fmt=:svg)

Here we see that oscillations remain, but become much noiser. Note, in
constructing the `JumpProblem` we could have used any of the SSAs that are part
of DiffEqJump instead of the `Direct` method, see the list of SSAs (i.e.
constant rate jump aggregators) in the
[documentation](http://docs.juliadiffeq.org/latest/types/jump_types.html#Constant-Rate-Jump-Aggregators-1).

---
## $\tau$-leaping Methods:
While SSAs generate exact realizations for stochastic chemical kinetics jump
process models, [$\tau$-leaping](https://en.wikipedia.org/wiki/Tau-leaping)
methods offer a performant alternative by discretizing in time the underlying
time-change representation of the stochastic process. The DiffEqJump package has
limited support for $\tau$-leaping methods in the form of the basic Euler's
method type approximation proposed by Gillespie. We can simulate a $\tau$-leap
approximation to the repressilator by using the  `RegularJump` representation of
the network to construct a `JumpProblem`:

In [ ]:
rjs = regularjumps(repressilator)
lprob = JumpProblem(dprob, Direct(), rjs)
lsol = solve(lprob, SimpleTauLeaping(), dt=.1)
plot(lsol, plotdensity=1000, fmt=:svg)

---
## Chemical Langevin Equation (CLE) Stochastic Differential Equation (SDE) Models:
At an intermediary physical scale between macroscopic ODE models and microscopic
stochastic chemical kinetic models lies the CLE, a SDE version of the model. The
SDEs add to each ODE above a noise term. As the repressilator has species that
get very close to zero in size, it is not a good candidate to model with the CLE
(where solutions can then go negative and become unphysical). Let's create a
simpler reaction network for a birth-death process that will stay non-negative:

In [ ]:
bdp = @reaction_network begin
  c₁, X --> 2X
  c₂, X --> 0
  c₃, 0 --> X
end c₁ c₂ c₃
p = (1.0,2.0,50.)
u₀ = [5.]
tspan = (0.,4.);

The corresponding Chemical Langevin Equation SDE is then

In [ ]:
latexify(bdp, noise=true, cdot=false)

where each $W_i(t)$ denotes an independent Brownian Motion. We can solve the CLE
SDE model by creating an `SDEProblem` and solving it similar to what we did for
ODEs above:

In [ ]:
# SDEProblem for CLE
sprob = SDEProblem(bdp, u₀, tspan, p)

# solve and plot, tstops is used to specify enough points 
# that the plot looks well-resolved
sol = solve(sprob, tstops=range(0., step=4e-3, length=1001))
plot(sol, fmt=:svg)

We again have complete freedom to select any of the
StochasticDifferentialEquations.jl SDE solvers, see the
[documentation](http://docs.juliadiffeq.org/latest/solvers/sde_solve.html).

---
## What information can be queried from the reaction_network:
The generated `reaction_network` contains a lot of basic information. For example
- `f=oderhsfun(repressilator)` is a function `f(du,u,p,t)` that given the current
  state vector `u` and time `t` fills `du` with the time derivatives of `u`
  (i.e. the right hand side of the ODEs).
- `jac=jacfun(repressilator)` is a function `jac(J,u,p,t)` that evaluates and
  returns the Jacobian of the ODEs in `J`. A corresponding Jacobian matrix of
  expressions can be accessed using the `jacobianexprs` function:

In [ ]:
latexify(jacobianexprs(repressilator), cdot=false)

- `pjac = paramjacfun(repressilator)` is a function `pjac(pJ,u,p,t)` that
  evaluates and returns the Jacobian, `pJ`, of the ODEs *with respect to the
  parameters*. This allows `reaction_network`s to be used in the
  DifferentialEquations.jl local sensitivity analysis package
  [DiffEqSensitivity](http://docs.juliadiffeq.org/latest/analysis/sensitivity.html).


By default, generated `ODEProblems` will be passed the corresponding Jacobian
function, which will then be used within implicit ODE/SDE methods. 

The [DiffEqBiological API
documentation](http://docs.juliadiffeq.org/latest/apis/diffeqbio.html) provides
a thorough description of the many query functions that are provided to access
network properties and generated functions. In DiffEqBiological Tutorial II
we'll explore the API.

---
## Getting Help
Have a question related to DiffEqBiological or this tutorial? Feel free to ask
in the DifferentialEquations.jl [Gitter](https://gitter.im/JuliaDiffEq/Lobby).
If you think you've found a bug in DiffEqBiological, or would like to
request/discuss new functionality, feel free to open an issue on
[Github](https://github.com/JuliaDiffEq/DiffEqBiological.jl) (but please check
there is no related issue already open). If you've found a bug in this tutorial,
or have a suggestion, feel free to open an issue on the [DiffEqTutorials Github
site](https://github.com/JuliaDiffEq/DiffEqTutorials.jl). Or, submit a pull
request to DiffEqTutorials updating the tutorial!

---